In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from core.s3_utils import S3_Bucket
import pandas as pd
import re
import plotly.express as px
import numpy as np

In [ ]:
from transform.raw_tss.main import get_raw_tss



# Load data from S3

In [ ]:
list_files = S3_Bucket().list_responses_keys_of_brand('stellantis')

In [ ]:
list_files

In [ ]:
vin_list = ["VR3UKZKWZPJ789947", "VR3UKZKW5RJ625386", "VR3UKZKWZPJ911720", "VR3UKZKWZPJ911696", "VR3UKZKWZPJ918307", "VR3UKZKWZPJ789946",
            "VR3UKZKWZPJ853581", "VR3UKZKWZPJ911694", "VR3UKZKW6RJ619807", "VR3UKZKWZPJ911695", "VR3UKZKWZPJ842705", "VR3UKZKWZPJ918303"]

df_stellantis = pd.DataFrame()
engine_df = pd.DataFrame()
electricty_df = pd.DataFrame()
df_final = pd.DataFrame()
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   if len(res) > 0:
      if res[0] in vin_list[12:]:
         data = S3_Bucket().read_json_file(file['key'])
         if data:
            vin = data.get('vin')
            odometer = pd.DataFrame(data.get('odometer', '')).rename(columns={'value': 'odometer'}).sort_values('datetime')
            odometer['datetime'] = pd.to_datetime(odometer['datetime'])
            battery_state = pd.DataFrame(data.get('engine', {}).get("battery", '')).sort_values('datetime')
            battery_state['datetime'] = pd.to_datetime(battery_state['datetime'])
            electricity =  data.get('electricity', {}).get("level", '')
            engine = data.get('engine', {}).get('contact', {})
            df = pd.merge(battery_state, odometer, on='datetime')
            if len(electricity) > 0:
               electricity = pd.DataFrame(electricity).rename(columns={'percentage': "batterie_percentage"}).sort_values('datetime')
               charging = pd.DataFrame(data.get("electricity").get('charging')).sort_values('datetime')
               electricity['datetime'] = pd.to_datetime(electricity['datetime'])
               charging['datetime'] = pd.to_datetime(charging['datetime'])
               autonomy = pd.DataFrame(data.get("electricity").get('residualAutonomy')).rename(columns={'value':'estimated_km'})
               autonomy['datetime'] = pd.to_datetime(autonomy['datetime'])
               elec = pd.merge(electricity, charging, on='datetime', how='outer').merge(autonomy, on='datetime', how='outer')
               electricty_df = pd.concat((electricty_df,elec ))
               
            
            if len(engine) > 0:
               
               engine = pd.DataFrame(engine).rename(columns={'value': 'contact'}).sort_values('datetime')
               engine['datetime'] = pd.to_datetime(engine['datetime'])
               engine_df = pd.concat((engine_df,engine ))
            
            df = df.sort_values('odometer')
            df_stellantis = pd.concat((df_stellantis, df))
         df_stellantis.drop(columns=['resistance', 'capacity'], inplace=True)      
         df_stellantis.reset_index(drop=True, inplace=True)   
         df_vin_test = electricty_df.merge(df_stellantis, on='datetime', how='outer').sort_values('datetime').drop_duplicates()
         df_vin_test['vin'] = vin
      # if len(df_vin_test) > 0:
      #    df_final = pd.concat((df_final, df_vin_test))


In [ ]:
df_stellantis = pd.DataFrame()
for index, file in list_files.iterrows():
   res = re.findall("[A-Z]\w+", file['key'])
   if len(res) > 0:
      if res[0] in vin_list:
         data = S3_Bucket().read_json_file(file['key'])
         if data:
            vin = data.get('vin')
            odometer = pd.DataFrame(data.get('odometer', '')).rename(columns={'value': 'odometer'}).sort_values('datetime')[["datetime","odometer"]]
            odometer['datetime'] = pd.to_datetime(odometer['datetime'])
            electricity =  data.get('electricity', {}).get("level", '')
            engine = data.get('engine', {}).get('contact', {})
            temp = pd.DataFrame(data.get('externalTemperature')).rename(columns={'value': 'external_temp'}).sort_values('datetime')[['external_temp', 'datetime']]
            temp['datetime'] = pd.to_datetime(temp['datetime'])
            df = pd.merge_asof(odometer, temp, on='datetime')
            
            if len(electricity) > 0:
               electricity = pd.DataFrame(electricity).rename(columns={'percentage': "soc"}).sort_values('datetime')
               charging = pd.DataFrame(data.get("electricity").get('charging')).sort_values('datetime')
               electricity['datetime'] = pd.to_datetime(electricity['datetime'])
               autonomy = pd.DataFrame(data.get("electricity").get('residualAutonomy'))[['datetime','value']].rename(columns={'value':'estimated_km'}).sort_values('datetime')
               autonomy['datetime'] = pd.to_datetime(autonomy['datetime'])
               charging['datetime'] = pd.to_datetime(charging['datetime'])
               elec = pd.merge_asof(electricity, charging, on='datetime')
               elec = pd.merge_asof(elec, autonomy, on='datetime')
               df =  pd.merge_asof(df, elec, on='datetime')

            if len(engine) > 0:
               engine = pd.DataFrame(engine).rename(columns={'value': 'contact'}).sort_values('datetime')
               engine['datetime'] = pd.to_datetime(engine['datetime'])
               df = pd.merge_asof(df, engine, on='datetime')
            
            df['vin'] = vin
            df = df.sort_values('odometer')
            df_stellantis = pd.concat((df_stellantis, df))
     
df_stellantis.reset_index(drop=True, inplace=True)


In [ ]:
df_stellantis.shape

In [ ]:
df_stellantis.isna().sum()

In [ ]:
df_stellantis.dropna(inplace=True)
df_stellantis.reset_index(drop=True, inplace=True)
df_stellantis.shape

In [ ]:
# add start date for each vin 
date_start_df = pd.DataFrame([['VR3UKZKWZPJ789947',"7-11-2023"],
['VR3UKZKW5RJ625386',"3-4-2024"],
['VR3UKZKWZPJ911720',"5-2-2024"],
['VR3UKZKWZPJ911696',"11-3-2024"],
['VR3UKZKWZPJ918307',"13-2-2024"],
['VR3UKZKWZPJ789946',"13-11-2023"],
['VR3UKZKWZPJ853581',"13-12-2023"],
['VR3UKZKWZPJ911694',"16-02-2024"],
['VR3UKZKW6RJ619807',"22-03-2024"],
['VR3UKZKWZPJ911695',"27-02-2024"],
['VR3UKZKWZPJ842705',"29-11-2023"],
['VR3UKZKWZPJ918303',"30-01-2024"],], columns=['vin', 'date_start'])
date_start_df['date_start'] = pd.to_datetime(date_start_df['date_start'],  dayfirst=True)

df_stellantis = df_stellantis.merge(date_start_df, on='vin')

In [ ]:
# show the vin whcich no data
set(vin_list) - set(df_stellantis.vin.unique())

# SoH computation

On suppose que l'autonomie est 400 mais ce n'est pas forcément vrai mais le nombre estimé de km est de + de 370 pour chacun des véhicules sauf VR3UKZKWZPJ918303
Et sur le site de PSA l'estimation du dernier 2008 est comprise entre 400 et 406 km ou 341-343 km. 

In [ ]:
# Max estimated km to be sure that we use 400km as reference for a new battery
df_stellantis.groupby('vin')['estimated_km'].max()

In [ ]:
# take only discharging status -> Pluged = False & status = Disconnected

In [ ]:
df_stellantis_discharge = df_stellantis[(df_stellantis['plugged']==False) & 
                        (df_stellantis['status'] == 'disconnected')].copy().reset_index(drop=True)

In [ ]:
# compenser l'autonomie d'une température froide -> optimal suposé vers 20*C d'après internet
df_stellantis_discharge['soh'] = df_stellantis_discharge.apply(lambda x: ((x['estimated_km'])  / (400*x['soc']/100)), axis=1)
px.imshow(df_stellantis_discharge[['soh', 'soc', 'odometer', 'estimated_km', 'external_temp']].corr().round(2), text_auto=True)


## Visualisation SoC versus nb Km estimé

In [ ]:
df_stellantis['range/soc'] = df_stellantis['estimated_km'] / df_stellantis['soc']

In [ ]:
px.scatter(df_stellantis, x='soc', y='range/soc', title='Variation of Estimated Kilometers per point of State of Charge (SoC)')

In [ ]:
px.scatter(df_stellantis, x='estimated_km', y='range/soc', title='Variation of Estimated Kilometers per point of State of Charge (SoC) ', trendline='ols')

We can clearly see that lower is the number of SoC point / estimated km lower is the ration between SoC point and  estimated km

## Focus on hight SoC

In [ ]:
df_stellantis_hight_soc = df_stellantis_discharge[df_stellantis_discharge['soc'] > 40].copy()

In [ ]:
df_stellantis_hight_soc.columns

### Visualisation estimated km


In [ ]:
px.scatter(df_stellantis_hight_soc, x='estimated_km', y='soh', trendline='ols')

### Visualisation SoC


In [ ]:
px.scatter(df_stellantis_hight_soc, x='soc', y='soh', trendline='ols')

## Première dépendance soc à étudier

In [ ]:
df_stellantis_hight_soc['soh_norm_soc'] = df_stellantis_hight_soc['soh'] / df_stellantis_hight_soc['soc']

In [ ]:
px.imshow(df_stellantis_hight_soc[['soh_norm_soc', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr().round(2), text_auto=True)


Divided by SoC does not remove the high corelation just invert it

### Adjust etim km by SOC

In [ ]:
def correction_faible_SoC_V2(km_estime, SoC, ref_points=[(78, 301), (96, 386)]):
    if SoC >= 50:
        return km_estime  
    
    SoC_high, km_high = ref_points[-1]
    SoC_low, km_low = ref_points[0]
    
    km_theorique = abs(km_low + (km_high - km_low) * ((SoC - SoC_low) / (SoC_high - SoC_low)))
    erreur = (km_theorique - km_estime) / km_theorique  
    correction_factor = min(0.9, 1 - erreur)  # Évite corrections extrêmes
    
    km_corrige = km_estime / correction_factor
    return max(km_corrige, km_theorique) 

In [ ]:
df_stellantis_discharge['estimated_km_adjusted_V2'] = df_stellantis_discharge.apply(lambda x: correction_faible_SoC_V2(x["estimated_km"], x["soc"]), axis=1)


In [ ]:
df_stellantis_discharge['estimated_km_adjusted_V2/soc'] = df_stellantis_discharge['estimated_km_adjusted_V2'] / df_stellantis['soc']


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=df_stellantis_discharge['estimated_km_adjusted_V2/soc'], x=df_stellantis_discharge['soc'], mode='markers', name='correction linéaire'))
fig.show()

Compute the SoH with the adjusted_V2 estimated km

In [ ]:
df_stellantis_discharge['soh_adj_V2'] = df_stellantis_discharge['estimated_km_adjusted_V2']  / ((400 * df_stellantis_discharge['soc']) /100)

In [ ]:
px.imshow(df_stellantis_discharge[['soh_adj_V2', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr().round(2), text_auto=True)


On garde la corrélation ave le SoC et le nombre de km estimé 

In [ ]:
px.scatter(df_stellantis_discharge, x='soc', y='soh_adj_V2')

In [ ]:
px.scatter(df_stellantis_discharge, x='estimated_km', y='soh_adj_V2')

la correction n'a pas d'effet on va essayer avec une correction non linéaire

## correction non linéaire

In [ ]:
from scipy.optimize import curve_fit
import numpy as np
def nonlinear_correction(SoC, a, b):
    return a * np.exp(b * SoC) 

# Ajustement de la courbe sur les valeurs de référence (71% -> 266 km et 96% -> 386 km)
ref_SoC = np.array([78, 96])
ref_km = np.array([301, 386])

params, _ = curve_fit(nonlinear_correction, ref_SoC, ref_km, p0=(1, 0.01))

def correction_faible_SoC_nl(km_estime, SoC):
    if SoC >= 50:
        return km_estime  # Pas de correction pour SoC élevés

    km_theorique = nonlinear_correction(SoC, *params)  # Estimation ajustée
    erreur = (km_theorique - km_estime) / km_theorique  
    correction_factor = max(0.9, 1 - erreur)  # Sécurité pour éviter surestimation

    km_corrige = km_estime / correction_factor
    return min(km_corrige, km_theorique)  # Évite de dépasser la théorie


In [ ]:
df_stellantis_discharge['estimated_km_adjusted_V3'] = df_stellantis_discharge.apply(lambda x: correction_faible_SoC_nl(x['estimated_km'], x['soc']), axis=1)

In [ ]:
df_stellantis_discharge['estimated_km_adjusted_V3/soc'] = df_stellantis_discharge['estimated_km_adjusted_V3'] / df_stellantis['soc']

In [ ]:
df_stellantis_discharge['soh_adj_V3'] = df_stellantis_discharge['estimated_km_adjusted_V3']  / ((400 * df_stellantis_discharge['soc']) /100)

In [ ]:
px.imshow(df_stellantis_discharge[['soh_adj_V3', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr().round(2), text_auto=True)


In [ ]:
px.scatter(df_stellantis_discharge, x='soc', y='soh_adj_V3')

## Use polynom to regularize

In [ ]:
import numpy as np

def correction_faible_SoC_V3(km_estime, SoC, ref_points=[(78, 301), (96, 386)]):
    
    SoC_vals, km_vals = zip(*ref_points)
    
    # Ajustement polynomial 
    poly_coeffs = np.polyfit(SoC_vals, km_vals, 2)  
    km_theorique = np.polyval(poly_coeffs, SoC)
    
    erreur = (km_theorique - km_estime) / km_theorique
    
    correction_factor = max(0.8, min(1.1, 1 - erreur))
    
    km_corrige = km_estime / correction_factor
    
    return max(km_corrige, km_theorique)  


In [ ]:
for vin in vin_list:
    print(vin)
    
    # Filter data once
    df_vin = df_stellantis_discharge[df_stellantis_discharge['vin'] == vin]
    df_vin['estimated_km/soc'] =  df_vin['estimated_km'] / df_vin['soc'] 
    # Calculate medians
    soc_med = df_vin['soc'].mean()
    range_med = df_vin['estimated_km'].mean()
    
    # Apply condition for soc_max and range_max
    df_filtered = df_vin[df_vin['estimated_km/soc'] <= 4]
    soc_max = df_filtered['soc'].max()
    range_max = df_filtered['estimated_km'].max()
    
    print(soc_max, soc_med, range_max, range_med)

    # Get index for updates
    index = df_vin.index

    # Apply correction function correctly using .loc[]
    df_stellantis_discharge.loc[index, 'range_corrected'] = df_vin.apply(
        lambda x: correction_faible_SoC_V3(
            x["estimated_km"], 
            x["soc"], 
            ref_points=[(soc_med, range_med), (soc_max, range_max)]
        ), axis=1
    )

In [ ]:
df_stellantis_discharge['soh_adj_vin'] = df_stellantis_discharge['range_corrected']  / ((400 * df_stellantis_discharge['soc']) /100)

In [ ]:
px.imshow(df_stellantis_discharge[['soh_adj_vin', 'soc', 'odometer', 'estimated_km', 'external_temp',]].corr().round(2), text_auto=True)


In [ ]:
px.scatter(df_stellantis_discharge, x='soc', y='soh_adj_vin', trendline='ols')

In [ ]:
df_stellantis_discharge[df_stellantis_discharge['soh_adj_vin']<1.05].groupby('vin')[['soh_adj_vin', 'soh_adj_V2']].mean()

In [ ]:
df_stellantis_discharge[df_stellantis_discharge['soh_adj_vin']<1.05].groupby('vin')[['soh_adj_vin', 'soh_adj_V2']].median()